**Authors**: Kazmer Nagy-Betegh, Clara Moreno Sanchez, Jasmine Zhang, Sophia Kalusche, Yingjin He, Abdullah Rehman

# AM13 Group Project

## Introduction

In [182]:
import gurobipy as gp
from gurobipy import GRB,quicksum
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
import sys
import csv
import math

## Loading the data 

In [196]:
# read electives data
courses_instances = pd.read_excel('data/CourseSessionsList MAM2022.xlsx')
courses_instances.head()

# clean rooms
courses_instances['Rooms'] = courses_instances['Rooms'].str.replace('Virtual Online', '')
courses_instances['Rooms'] = courses_instances['Rooms'].str.replace(' ', '')
courses_instances['Rooms'] = courses_instances['Rooms'].str.replace('SOC', '')
courses_instances['Rooms'] = courses_instances['Rooms'].str.replace('SusxPlc', '')
courses_instances['Rooms'] = courses_instances['Rooms'].str.replace(',', '')

courses_instances[courses_instances["Session Type"] == "Lecture"].Rooms.unique()

courses_instances['Rooms'] = courses_instances['Rooms'].str.replace('PB-LAB', 'PBLab')
courses_instances['Rooms'] = courses_instances['Rooms'].str.replace('NBLT12', 'LT12')
courses_instances['Rooms'] = courses_instances['Rooms'].str.replace('WLT', 'LT12')
courses_instances[courses_instances["Session Type"] == "Lecture"].Rooms.unique()

# read capacity data
capacity = pd.read_excel('data/capacity.xlsx')
capacity.drop(['Unnamed: 0'], axis=1, inplace=True)
rooms = capacity.LT.to_list()
rooms[0] = 'LT1'
courses_instances_lecture = courses_instances[(courses_instances["Session Type"] == "Lecture")]

# drop rows with rooms not in capacity
courses_instances_lecture = courses_instances_lecture[courses_instances_lecture["Rooms"].isin(rooms)]

courses_instances_lecture.Rooms.unique()
# create schedule table for each day
rooms = capacity.LT.to_list()
rooms[0] = 'LT1'
scheduling_table = pd.DataFrame(columns=['day', 'time']+rooms)

days = courses_instances["Session Date"].unique()

days_24h = []
# create 15 min increment for each day
for d in days:
    day_times = pd.date_range(start=d+str(" 00:00"), end=d+str(" 23:59"), freq='15min')
    days_24h = days_24h + day_times.to_list()
scheduling_table['day'] = days_24h
scheduling_table['time'] = scheduling_table['day'].apply(lambda x: x.strftime('%H:%M'))
scheduling_table.day = scheduling_table.day.apply(lambda x: x.strftime('%Y-%m-%d'))
scheduling_table.sort_values(by=['day', "time"], inplace=True)

# replace NaN with 0
scheduling_table.fillna(0, inplace=True)

scheduling_table.head()
# mark column if it is blocked

for i in range(courses_instances_lecture.shape[0]):
    date = courses_instances_lecture.iloc[i][ "Session Date"]
    start_time = courses_instances_lecture.iloc[i][ "Start Time"]
    end_time = courses_instances_lecture.iloc[i][ "End Time"]
    room = courses_instances_lecture.iloc[i][ "Rooms"]
    
    scheduling_table.loc[(scheduling_table.day == date) & (scheduling_table.time >= start_time) & (scheduling_table.time <= end_time), room] = 1
# find all lt1 equal to 1

scheduling_table.loc[scheduling_table.LT1 == 1]

,day,time,LT1,LT2,LT3,LT4,LT5,LT6,LT7,LT9,...,LT15,LT16,LT17,LT18,LT19,PLG01,PBLab,Trans,WLT,RG06
417,2021-09-13,08:15,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
418,2021-09-13,08:30,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
419,2021-09-13,08:45,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
420,2021-09-13,09:00,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
421,2021-09-13,09:15,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16378,2022-06-08,14:30,1,0,0,0,0,0,1,0,...,1,0,1,0,0,0,0,0,0,0
16379,2022-06-08,14:45,1,0,0,0,0,0,1,0,...,1,0,1,0,0,0,0,0,0,0
16380,2022-06-08,15:00,1,0,0,0,0,0,1,0,...,1,0,1,0,0,0,0,0,0,0
16381,2022-06-08,15:15,1,0,0,0,0,0,1,0,...,1,0,1,0,0,0,0,0,0,0


## Create constraints

In [9]:
# gurobi model
m = gp.Model("Room_Scheduling")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-01-01


In [ ]:
# create function to set booked binary variables
def create_rooms_constraints(model, rooms):
    for room in rooms:
        model.addConstr(quicksum(scheduling_table[room]["Time"]) <= 1)